In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random

DS= pd.read_csv("boston_2019.csv", sep= ",")
SD= pd.read_csv("start_station_data_2019.csv", sep= ",")

In [4]:
SD.head()

,Unnamed: 0,start_time,end_time,start_station_id,end_station_id,start_station_name,bike_id,user_type,duration_m,District,start_coo
0,0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.336,80,179,MIT Stata Center at Vassar St / Main St,3689,Subscriber,6.192300,Cambridge,"(42.3621, -71.0912)"
1,1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.880,117,189,Binney St / Sixth St,4142,Subscriber,4.411633,Cambridge,"(42.3661, -71.0863)"
2,2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.273,68,96,Central Square at Mass Ave / Essex St,1628,Subscriber,7.644550,Cambridge,"(42.3651, -71.1031)"
3,3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.426,89,334,Harvard Law School at Mass Ave / Jarvis St,2969,Subscriber,6.080917,Cambridge,"(42.379, -71.1199)"
4,4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.701,73,367,Harvard Square at Brattle St / Eliot St,3469,Subscriber,11.353950,Cambridge,"(42.3732, -71.1209)"


In [5]:
station_list = DS["start_station_id"].unique()
selected_station_id = random.choice(station_list)
print(selected_station_id)

40


In [6]:
c = pd.DataFrame(SD["start_coo"].str.split(expand= True))

c = c.rename(columns={0: "long", 1: "lat"})

c["long"] = c["long"].str.strip("(,")
c["lat"] = c["lat"].str.strip(",)")

c.info()

c["lat"] = c["lat"].astype(float)
c["long"] = c["long"].astype(float)

c["Coordinates"] = list(zip(c["long"].round(4),c["lat"].round(4)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522771 entries, 0 to 2522770
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   long    object
 1   lat     object
dtypes: object(2)
memory usage: 38.5+ MB


In [7]:
SD["Coordinates"] = c["Coordinates"]

In [8]:
SD.drop(columns= "Unnamed: 0")

,start_time,end_time,start_station_id,end_station_id,start_station_name,bike_id,user_type,duration_m,District,start_coo,Coordinates
0,2019-01-01 00:09:13.798,2019-01-01 00:15:25.336,80,179,MIT Stata Center at Vassar St / Main St,3689,Subscriber,6.192300,Cambridge,"(42.3621, -71.0912)","(42.3621, -71.0912)"
1,2019-01-01 00:33:56.182,2019-01-01 00:38:20.880,117,189,Binney St / Sixth St,4142,Subscriber,4.411633,Cambridge,"(42.3661, -71.0863)","(42.3661, -71.0863)"
2,2019-01-01 00:41:54.600,2019-01-01 00:49:33.273,68,96,Central Square at Mass Ave / Essex St,1628,Subscriber,7.644550,Cambridge,"(42.3651, -71.1031)","(42.3651, -71.1031)"
3,2019-01-01 00:43:32.571,2019-01-01 00:49:37.426,89,334,Harvard Law School at Mass Ave / Jarvis St,2969,Subscriber,6.080917,Cambridge,"(42.379, -71.1199)","(42.379, -71.1199)"
4,2019-01-01 00:49:56.464,2019-01-01 01:01:17.701,73,367,Harvard Square at Brattle St / Eliot St,3469,Subscriber,11.353950,Cambridge,"(42.3732, -71.1209)","(42.3732, -71.1209)"
...,...,...,...,...,...,...,...,...,...,...,...
2522766,2019-12-31 23:52:10.728,2020-01-01 00:18:13.418,22,22,South Station - 700 Atlantic Ave,4503,Customer,26.044833,Boston,"(42.3522, -71.0555)","(42.3522, -71.0555)"
2522767,2019-12-31 23:52:14.927,2020-01-01 00:29:45.821,22,22,South Station - 700 Atlantic Ave,5326,Customer,37.514900,Boston,"(42.3522, -71.0555)","(42.3522, -71.0555)"
2522768,2019-12-31 23:54:58.391,2020-01-01 00:01:35.479,386,228,Sennott Park Broadway at Norfolk Street,4483,Subscriber,6.618133,NaN,NaN,"(nan, nan)"
2522769,2019-12-31 23:55:54.544,2020-01-01 00:01:55.427,47,98,Cross St at Hanover St,3955,Subscriber,6.014717,Boston,"(42.3628, -71.0561)","(42.3628, -71.0561)"


In [9]:
test = SD["start_station_id"]

In [10]:
SD = SD[SD["start_coo"].notna()]

In [ ]:
#define new map

StartHM = folium.Map(location=(42.3611, -71.0571),tiles='OpenStreetMap',zoom_start=12, control_scale=True, max_zoom=20)

# add heat map

StartHM.add_child(plugins.HeatMap(SD["Coordinates"], radius=20))

StartHM